In [10]:
import pandas as pd
import numpy as np
import os
import glob

In [11]:
output_file = "demo_output.xlsx"
df = pd.read_excel(output_file)

In [12]:
df.head()

,date,time,group,subject,build,blocknum,trialnum,blockcode,trialcode,pretrialpause,...,stimulusonset1,stimulusnumber2,stimulusitem2,stimulusonset2,stimulusnumber3,stimulusitem3,stimulusonset3,stimulusnumber4,stimulusitem4,stimulusonset4
0,82416,13:11:41,1,1,5.0.1.0,2,2,prac_prescene,prac_prescene,0,...,6,1,images/keyboard_empty.png,6,1,indoor or outdoor,6,1,images/1188.jpg,6
1,82416,13:11:41,1,1,5.0.1.0,2,5,prac_prescene,prac_prescene,0,...,6,1,images/keyboard_empty.png,6,1,indoor or outdoor,6,2,images/1189.jpg,6
2,82416,13:11:41,1,1,5.0.1.0,2,8,prac_prescene,prac_prescene,0,...,7,1,images/keyboard_empty.png,7,1,indoor or outdoor,7,3,images/1190.jpg,7
3,82416,13:11:41,1,1,5.0.1.0,2,11,prac_prescene,prac_prescene,0,...,7,1,images/keyboard_empty.png,7,1,indoor or outdoor,7,4,images/1187.jpg,7
4,82416,13:11:41,1,1,5.0.1.0,2,14,prac_prescene,prac_prescene,0,...,7,1,images/keyboard_empty.png,7,1,indoor or outdoor,7,5,images/1190.jpg,7


## DROP COLUMNS

In [13]:
# Drop unneeded columns
# stimulusitem4 contains Pre-Scene Stim.

columns_to_remove = ["pretrialpause", "posttrialpause", "trialduration", "trialtimeout", "stimulusonset1",
                     "stimulusonset2", "stimulusonset3", "stimulusonset4", "stimulusitem1", "stimulusitem2", "stimulusitem3",
                     "group", "build", "correct", "stimulusnumber1", "stimulusnumber2", "stimulusnumber3", "stimulusnumber4", 
                     "trialcode"]
df = df.drop(columns_to_remove, axis=1)

In [14]:
df.head()

,date,time,subject,blocknum,trialnum,blockcode,response,latency,stimulusitem4
0,82416,13:11:41,1,2,2,prac_prescene,31,883,images/1188.jpg
1,82416,13:11:41,1,2,5,prac_prescene,30,1163,images/1189.jpg
2,82416,13:11:41,1,2,8,prac_prescene,31,1002,images/1190.jpg
3,82416,13:11:41,1,2,11,prac_prescene,31,1820,images/1187.jpg
4,82416,13:11:41,1,2,14,prac_prescene,0,2000,images/1190.jpg


## REMOVE VALUES

In [15]:
# Replace no responses which are represented with 0 as NaN
df['response'] = df['response'].replace(0, np.nan)

In [16]:
df.head()

,date,time,subject,blocknum,trialnum,blockcode,response,latency,stimulusitem4
0,82416,13:11:41,1,2,2,prac_prescene,31,883,images/1188.jpg
1,82416,13:11:41,1,2,5,prac_prescene,30,1163,images/1189.jpg
2,82416,13:11:41,1,2,8,prac_prescene,31,1002,images/1190.jpg
3,82416,13:11:41,1,2,11,prac_prescene,31,1820,images/1187.jpg
4,82416,13:11:41,1,2,14,prac_prescene,NaN,2000,images/1190.jpg


## CHANGE VALUES

In [17]:
# If there is no response, latency should be NaN, not the trial length
df.loc[df['response'].isnull(), 'latency'] = np.nan


# Remove the file path for the pre-scene stim.
df = df.replace("images/", "", regex=True).replace(".jpg", "", regex=True)

# 30 represents the letter 'A'. 31 represents the letter 'B'. Map it use values 0 and 1 for A and B
df['response'] = df['response'].replace(
    {30: 0, 31: 1, 57: "SPACE"})

In [18]:
df.head()

,date,time,subject,blocknum,trialnum,blockcode,response,latency,stimulusitem4
0,82416,13:11:41,1,2,2,prac_prescene,1,883.0,1188
1,82416,13:11:41,1,2,5,prac_prescene,0,1163.0,1189
2,82416,13:11:41,1,2,8,prac_prescene,1,1002.0,1190
3,82416,13:11:41,1,2,11,prac_prescene,1,1820.0,1187
4,82416,13:11:41,1,2,14,prac_prescene,NaN,NaN,1190


In [19]:
# Get the name of the different blocks
# There will be a seperate output file for each block
block_types = df['blockcode'].drop_duplicates().tolist()
block_types

[u'prac_prescene',
 u'survey',
 u'b1_prescene',
 u'b2_prescene',
 u'b3_prescene',
 u'demographics_block',
 u'worker_id_block']

In [20]:
if not os.path.exists("block_outputs"):
    os.mkdir("block_outputs")

for block in block_types:
    
    print block
    
    # Only get the rows for the current block.
    block_df = df.loc[df['blockcode'] == block]
    
    # Blockcode column is not needed anymore
    block_df = block_df.drop("blockcode", axis=1)

    # Only clean the presce
    if "prescene" in block:
        
        # Change the column name to the appropriate name
        block_df = block_df.rename(columns={'stimulusitem4': 'PreScene'})
        
        # Set the correct block number                
        block_df['blocknum'] = block[1]
        
        # Set the correct trial numbers
        block_df["trialnum"] = [j+1 for j in range(block_df['trialnum'].size)]

    block_df.to_csv('block_outputs/%s.csv' % block, index=False)

prac_prescene
survey
b1_prescene
b2_prescene
b3_prescene
demographics_block
worker_id_block
